### Dashboard com dados diários do preço do Bitcoin


In [5]:
import requests
import pandas as pd

In [18]:
!pip install apache-airflow

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
    --------------------------------------- 0.3/13.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.4 MB 670.4 kB/s eta 0:00:20
   - -------------------------------------- 0.5/13.4 MB 670.4 kB/s eta 0:00:20
   -- ------------------------------------- 0.8/13.4 MB 780.2 kB/s eta 0:00:17
   --- ------------------------------------ 1.0/13.4 MB 867.1 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/13.4 MB 867.1 kB/s eta 0:00:15
   --- ------------------------------------ 1.3/13.4 MB 771.0 kB/s eta 0:00:16
   ---- ----------------------------------- 1.6/13.4 MB 806.6 kB/s eta 0:00:15
   ---- ----------------------------------- 1.6/13.4 MB 806.6 kB/s eta 0:00:15
   ----- ---------------------------------- 1.8/13.4 MB 


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!python -c "from airflow import DAG; print('Airflow instalado com sucesso')"

Airflow instalado com sucesso


C:\Users\claud\anaconda3\Lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


In [6]:
# Para Coletar Dados do Preço do Bitcoin vou usar uma API da CoinGecko para obter os dados históricos do Bitcoin(Tbm poderia ser a CoinMarketCap) 

def get_bitcoin_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        "vs_currency": "usd",
        "days": "180",  # Últimos 6 meses
        "interval": "daily"
    }
    response = requests.get(url, params=params)
    data = response.json()
    prices = data["prices"]
    
    # Transforma os dados em um DataFrame
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    
    return df

In [9]:
get_bitcoin_data()

,price
timestamp,
2024-08-10 00:00:00,60912.588533
2024-08-11 00:00:00,60887.708616
2024-08-12 00:00:00,58804.234500
2024-08-13 00:00:00,59350.074333
2024-08-14 00:00:00,60601.223178
...,...
2025-02-02 00:00:00,100674.787625
2025-02-03 00:00:00,97568.316530
2025-02-04 00:00:00,101466.860666


In [22]:
# Criar uma DAG no Apache Airflow que será responsável por:
# 1.Coletar os dados diários do Bitcoin, 
# 2.Salvar os dados em um arquivo CSV ou em um banco de dados,
# 3. Gerar um dashboard no Looker Studio

In [31]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
import requests


In [33]:
# Configuração da DAG
default_args = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": datetime(2023, 1, 1),
    "retries": 1,
    "retry_delay": timedelta(minutes=5),
}

dag = DAG(
    "bitcoin_price_dashboard",
    default_args=default_args,
    description="DAG para coletar dados do Bitcoin e gerar dashboard",
    schedule="@daily",  # Substituindo 'schedule_interval' por 'schedule'
)

In [35]:
# Tarefa para coletar dados
fetch_data_task = PythonOperator(
    task_id="fetch_bitcoin_data",
    python_callable=get_bitcoin_data,
    dag=dag,
)

In [37]:
# Tarefa para gerar dashboard (substitua por sua lógica)
def generate_dashboard():
    # Aqui você pode integrar com o Looker Studio ou outra ferramenta
    print("Dashboard gerado com sucesso!")

generate_dashboard_task = PythonOperator(
    task_id="generate_dashboard",
    python_callable=generate_dashboard,
    dag=dag,
)

In [39]:
# Define a ordem das tarefas
fetch_data_task >> generate_dashboard_task

<Task(PythonOperator): generate_dashboard>